In [1]:
import sys
sys.path.append('..')
import gluonbook as gb
import mxnet as mx
from mxnet import nd, gluon, init
from mxnet.gluon import nn

def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(channels=num_channels,
                          kernel_size=3, 
                          padding=1, 
                          activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2))
    return blk

def vgg(conv_arch, num_outputs):
    net = nn.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(
            nn.Dense(512, activation='relu'),
            nn.Dropout(0.5),
            nn.Dense(512, activation='relu'),
            nn.Dropout(0.5),
            nn.Dense(num_outputs),
            )
    return net


In [2]:
conv_arch = ((1,64), (1,128), (2,256), (2,512), (2,512))
net = vgg(conv_arch, 1000)
net.initialize()

X = nd.random.uniform(shape=(33,3,224,224))

for blk in net:
    X = blk(X)
    print(blk.name, ' shape:\t', X.shape)

sequential1  shape:	 (33, 64, 112, 112)
sequential2  shape:	 (33, 128, 56, 56)
sequential3  shape:	 (33, 256, 28, 28)
sequential4  shape:	 (33, 512, 14, 14)
sequential5  shape:	 (33, 512, 7, 7)
dense0  shape:	 (33, 512)
dropout0  shape:	 (33, 512)
dense1  shape:	 (33, 512)
dropout1  shape:	 (33, 512)
dense2  shape:	 (33, 1000)


In [3]:
ratio =  2
small_conv_arch = [(pair[0], int(pair[1]/ratio)) for pair in conv_arch]
net = vgg(small_conv_arch, 10)

In [4]:
ctx = gb.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.05})

train_data, test_data = gb.load_data_fashion_mnist(batch_size=90, resize=224)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=100)

C:\Program Files\Anaconda3\envs\ML\lib\site-packages\mxnet\gluon\data\vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
C:\Program Files\Anaconda3\envs\ML\lib\site-packages\mxnet\gluon\data\vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


training on  gpu(0)
epoch 1, loss 0.8709, train acc 0.676, test acc 0.847, time 279.6 sec
epoch 2, loss 0.4291, train acc 0.847, test acc 0.879, time 272.4 sec
epoch 3, loss 0.3478, train acc 0.875, test acc 0.899, time 272.3 sec
epoch 4, loss 0.3004, train acc 0.891, test acc 0.909, time 272.2 sec
epoch 5, loss 0.2686, train acc 0.903, test acc 0.914, time 272.4 sec
epoch 6, loss 0.2465, train acc 0.911, test acc 0.921, time 272.7 sec
epoch 7, loss 0.2252, train acc 0.919, test acc 0.923, time 272.5 sec
epoch 8, loss 0.2097, train acc 0.924, test acc 0.922, time 272.6 sec
epoch 9, loss 0.1940, train acc 0.930, test acc 0.929, time 272.5 sec
epoch 10, loss 0.1786, train acc 0.935, test acc 0.930, time 272.3 sec
epoch 11, loss 0.1629, train acc 0.939, test acc 0.928, time 272.4 sec
epoch 12, loss 0.1539, train acc 0.943, test acc 0.932, time 272.4 sec
epoch 13, loss 0.1404, train acc 0.949, test acc 0.933, time 272.5 sec
epoch 14, loss 0.1314, train acc 0.952, test acc 0.933, time 272.4